In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from task_decomposition.analysis.comparisons import (subtask_similarity,
                                                     extract_subtask_from_groundtruth_file,
                                                     extract_subtask_from_LLM_output_file,
                                                     semantic_distance)
from task_decomposition.paths import DATA_GT_TXT_PATH, LLM_OUTPUT_PATH

START_STEP_IDX = 0
END_STEP_IDX = 1
DESCRIPTION_IDX = 2

### Trajectory Visualization Functions

In [2]:
def format_string(s: list) -> str:
    """
    Add a newline after every 3 words in the list
    """
    N_WORDS_PER_LINE = 3
    x = s.split(" ")
    s_format = []
    for i in range(len(x)):
        s_format.append(x[i])
        s_format.append("\n") if i % N_WORDS_PER_LINE == 0 else None
    return " ".join(s_format)

# Redefine the add_intervals_with_colors function to position the text at the edges
def add_intervals_with_colors(intervals, row, color_map, ax, angle):
    for idx, (start, end, description) in enumerate(intervals):
        color = color_map[0]
        rect_width = end - start if end - start > 0 else 1
        rect = mpatches.Rectangle(
            (start, row), rect_width, 0.8, edgecolor="black", facecolor=color
        )
        ax.add_patch(rect)

        # Position the text at the top and bottom edges of the bars
        if row == 1:  # For the top bar, start the text at the top edge
            text_y = row + 0.8
            va = "bottom"
        else:  # For the bottom bar, start the text at the bottom edge
            text_y = row
            va = "top"

        # Place the text at a 45 degree angle for actual and -45 for predicted
        ax.text(
            (start+end) / 2,
            text_y,
            format_string(description),
            ha="left",
            va=va,
            fontsize="large",
            color="black",
            rotation=angle,
            rotation_mode="anchor",
        )

def plot_trajectory_decomposition(actual, predicted, title, ax=None):
    color_map = ['green', 'blue', 'purple', 'orange', 'red', 'yellow', 'pink', 'brown', 'grey', 'cyan']

    # preprocess so that each subtask in predicted matches the maximum semantic score in actual
    # predicted_color_map = []
    # for p in predicted:
    #     max_score = -1
    #     max_idx = 0
    #     for idx, a in enumerate(actual):
    #         score = semantic_distance(a[2], p[2])
    #         if score >= max_score:
    #             max_score = score
    #             max_idx = idx
    #     predicted_color_map.append(color_map[max_idx])

    # Add actual intervals with color map and angle the text at 45 degrees
    add_intervals_with_colors(actual, 1, color_map, ax, angle=35)

    # Add predicted intervals with color map and angle the text at -45 degrees
    add_intervals_with_colors(predicted, 0, color_map, ax, angle=-35)

    # Set the limits, labels, and title
    ax.set_xlim(-2, max(actual[-1][END_STEP_IDX], predicted[-1][END_STEP_IDX])+5)
    ax.set_ylim(-7, 12)
    ax.set_yticks([0.4, 1.4])
    ax.set_yticklabels(["Predicted", "Ground Truth"], fontsize=12)
    ax.set_xlabel("Step Number")
    ax.set_title(title, fontsize=14)

    plt.tight_layout()
    plt.show()

### Visualize Individual Trajectories

In [6]:
## visualize individual trajectory decompositions
llm_model = "gemini-pro-vision"
input_mode = "video"
env_name = "Lift"
runid = "Lift_20240220-124617_0"

## Extract the groundtruth and LLM output files
gt_file = DATA_GT_TXT_PATH(env_name)+f"/{runid}_gt.txt"
llm_file = LLM_OUTPUT_PATH(llm_model, input_mode, env_name)+f"/{runid}.json" 

gt_subtasks = extract_subtask_from_groundtruth_file(filepath=gt_file)
llm_subtasks = extract_subtask_from_LLM_output_file(filepath=llm_file,
                                                    llm_model=llm_model)

print(f"Ground Truth Subtasks: {gt_subtasks}")
print(f"LLM Subtasks: {llm_subtasks}")

similarity = subtask_similarity(gt_subtasks, llm_subtasks)
print(f"Similarity: {similarity}")

# # create new figure
fig, ax = plt.subplots(figsize=(14, 4))
title = f"{env_name} Environment, Total{similarity['total']:.2f}, Temporal: {similarity['temporal']:.2f}, Semantic Similarity: {similarity['semantic']:.2f}"
plot_trajectory_decomposition(actual=gt_subtasks,
                              predicted=llm_subtasks, 
                              title=f"{llm_model} | {input_mode} | {runid} | {similarity}", 
                              ax=ax)


TypeError: 'str' object is not callable

### Compute Similarity for one Trajectory

In [5]:
runid = "Stack_20240220-130911_22"
llm_model = "video-llava"
input_mode = "video"
env_name = "Stack"
gt_file = DATA_GT_TXT_PATH+f"/{runid}_gt.txt"
llm_file = LLM_OUTPUT_PATH(llm_model, input_mode, env_name)+f"/{runid}.json"
gt_subtasks = extract_subtask_from_groundtruth_file(gt_file)
llm_subtasks = extract_subtask_from_LLM_output_file(filepath=llm_file, 
                                                    llm_model=llm_model)
print(f"Ground Truth Subtasks: {gt_subtasks}")
print(f"LLM Subtasks: {llm_subtasks}")
similarity = subtask_similarity(gt_subtasks, llm_subtasks)
print(similarity)

Ground Truth Subtasks: [(0, 10, 'Move to above Cube A'), (11, 31, 'Move directly down to Cube A'), (32, 33, 'Grasp Cube A'), (34, 47, 'Vertically pick up Cube A'), (48, 66, 'Align Cube A with Cube B'), (67, 73, 'Move Cube A vertically down to Cube B'), (74, 77, 'Release Cube A onto Cube B'), (78, 79, 'Return Home')]
LLM Subtasks: [[0, 9, 'move to above object'], [9, 10, 'pick up the green object'], [10, 11, 'move to above object'], [11, 12, 'drop the green object'], [12, 13, 'move to above object'], [13, 14, 'pick up the red object'], [14, 15, 'drop the red object'], [15, 16, 'move to above object'], [16, 17, 'pick up the green object'], [17, 18, 'drop the green object'], [18, 19, 'move to above object'], [19, 20, 'pick up the red object'], [20, 21, 'drop the red object'], [21, 22, 'move to above object'], [22, 23, 'pick up the green object'], [23, 24, 'drop the green object'], [24, 25, 'move to above object'], [25, 26, 'pick up the red object'], [26, 27, 'drop the red object'], [27, 2

### Compute Similarity Metrics in batch by ENV_NAME, INPUT_MODE, LLM_MODEL

In [10]:
def compute_batch_similarity(env_name: str,
                             input_mode: str,
                             llm_model: str,
                             SAVE_OUTPUT: bool = True,
                             verbose: bool = True) -> None:
    """
    Compute the similarity between groundtruth and gpt output files for a given environment.
    """
    ## Ghetto way to extract files for now :(
    gt_files_relative_path = [f for f in os.listdir(DATA_GT_TXT_PATH) if env_name in f]
    gt_files= [os.path.join(DATA_GT_TXT_PATH, file) for file in gt_files_relative_path]
    gt_files = sorted(gt_files)

    llm_files_path = LLM_OUTPUT_PATH(llm_model, input_mode, env_name)
    llm_files = os.listdir(llm_files_path)
    llm_files = [os.path.join(llm_files_path, file) for file in llm_files]
    llm_files = sorted(llm_files)

    assert len(gt_files) == len(llm_files), f"The number of groundtruth files {len(gt_files)} and gpt output files {len(llm_files)} are not the same."

    n_runs = len(gt_files)
    output = {"temporal":np.full(n_runs, np.nan), 
              "semantic":np.full(n_runs, np.nan), 
              "total":np.full(n_runs, np.nan)}

    for idx, (gt_file, llm_file) in enumerate(zip(gt_files, llm_files)):
        # make sure we are comparing the same run
        gt_runid = gt_file.split('/')[-1].split('_gt')[0]
        llm_runid = llm_file.split('/')[-1].split('.json')[0]
        assert gt_runid==llm_runid, f"The groundtruth file {gt_file} and llm output file {llm_file} are not from the same run."

        # extract subtasks from groundtruth and gpt output files
        gt_subtasks = extract_subtask_from_groundtruth_file(gt_file)
        llm_subtasks = extract_subtask_from_LLM_output_file(filepath=llm_file, 
                                                            llm_model=llm_model)

        if llm_subtasks == []:    # error with processing the file
            print(f"LLM subtask for {llm_runid} is empty")
            continue
        # compare the subtasks
        similarity = subtask_similarity(gt_subtasks, llm_subtasks)
        output['temporal'][idx]=similarity["temporal"]
        output['semantic'][idx]=similarity["semantic"]
        output['total'][idx]=similarity["total"]
        if verbose:
            print(f"Similarity between {gt_runid} and {llm_runid} is {similarity['total']:.2f}")

    # Clean the output in case nans are present
    output['temporal'] = output['temporal'][~np.isnan(output['temporal'])]
    output['semantic'] = output['semantic'][~np.isnan(output['semantic'])]
    output['total'] = output['total'][~np.isnan(output['total'])]

    if SAVE_OUTPUT:
        np.savez(f"{LLM_OUTPUT_PATH(llm_model, input_mode, env_name)}_results.npz", **output)

    print(f"Number of runs: {n_runs}, Number of valid runs: {len(output['total'])}")
    print(f"Average Similarity: {np.mean(output['total']):.2f}, Average Temporal Similarity: {np.mean(output['temporal']):.2f}, Average Semantic Similarity: {np.mean(output['semantic']):.2f}")


env_names = ["Stack", "Lift", "Door", "PickPlace"]
input_mode = "text"
llm_model = "gpt-4-vision-preview"
for env_name in env_names:
    compute_batch_similarity(env_name=env_name,
                            input_mode=input_mode,
                            llm_model=llm_model,
                            SAVE_OUTPUT=True)


Similarity between Stack_20240220-130620_0 and Stack_20240220-130620_0 is 0.81
Similarity between Stack_20240220-130630_1 and Stack_20240220-130630_1 is 0.81
Similarity between Stack_20240220-130638_2 and Stack_20240220-130638_2 is 0.79
Similarity between Stack_20240220-130646_3 and Stack_20240220-130646_3 is 0.83
Similarity between Stack_20240220-130653_4 and Stack_20240220-130653_4 is 0.79
Similarity between Stack_20240220-130701_5 and Stack_20240220-130701_5 is 0.82
Similarity between Stack_20240220-130710_6 and Stack_20240220-130710_6 is 0.80
Similarity between Stack_20240220-130718_7 and Stack_20240220-130718_7 is 0.78
Similarity between Stack_20240220-130726_8 and Stack_20240220-130726_8 is 0.81
Similarity between Stack_20240220-130734_9 and Stack_20240220-130734_9 is 0.77
Similarity between Stack_20240220-130741_10 and Stack_20240220-130741_10 is 0.83
Similarity between Stack_20240220-130749_11 and Stack_20240220-130749_11 is 0.74
Similarity between Stack_20240220-130757_12 and 

In [ ]:
# sns.histplot(data=[temporal, semantic, total], legend=True, kde=True, bins=25)
# env_name = "Door"
# query_type = "video"
# npresultspath = GPT_OUTPUT_PATH+f"/{query_type}/{env_name}" 
# similarity = np.load(f"{npresultspath}_results.npz", allow_pickle=True)
N_BINS = 20
def make_histogram_grid(env_names: list, input_modes: list):
    """
    Create a grid of histograms for the similarity scores between groundtruth and gpt output files.
    Each histogram will show the distribution of the similarity scores for a given environment and query type.
    """
    n_rows = len(input_modes)
    n_cols = len(env_names)
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(25, 5*n_rows))
    for i, env_name in enumerate(env_names):
        for j, input_mode in enumerate(input_modes):
            npresultspath = LLM_OUTPUT_PATH(llm_model, input_mode, env_name)
            similarity = np.load(f"{npresultspath}_results.npz", allow_pickle=True)
            temporal = similarity['temporal']
            semantic = similarity['semantic']
            ax = axs[j, i] if n_rows > 1 else axs[i]
            sns.histplot(data=[temporal, semantic], legend=True, bins=N_BINS, ax=ax)
            ax.legend(["Temporal", "Semantic"], loc='upper left')
            ax.set_xlabel('Similarity')
            ax.set_ylabel('Frequency')
            ax.set_xlim(0, 1)
            input_mode = "Text and Video" if input_mode=="textvideo" else input_mode.capitalize()
            ax.set_title(f"{llm_model} {input_mode} | {env_name}, ")

llm_model = "gemini-pro"
envs = ["Lift", "Door", "Stack", "PickPlace"]
# input_modes = ["textvideo", "text", "video"]
input_modes = ["text"]
make_histogram_grid(envs, input_modes)


### Make the rows for the table (LLM_NAME, ENV_NAME, INPUT_MODE, SIMILARITY_METRIC, SIMILARITY_VALUE)

In [ ]:
# Make the big table of all the similarity results
llm_name = "gemini-pro-vision"
# query_types = ["textvideo", "text" , "video"]
query_types = ["video"]
# env_names = ["Lift", "Door", "Stack", "PickPlace"]
env_names = ["Lift", "Stack"]
for query_type in query_types:
    print(f"{query_type}: ")
    for env_name in env_names:
        npresultspath =  LLM_OUTPUT_PATH(llm_model, input_mode, env_name)
        similarity = np.load(f"{npresultspath}_results.npz", allow_pickle=True)
        total_mean, total_std = np.mean(similarity['total']), np.std(similarity['total'])
        print(f" {env_name}", end="")
        temporal_mean, temporal_std = np.mean(similarity['temporal']), np.std(similarity['temporal'])
        semantic_mean, semantic_std = np.mean(similarity['semantic']), np.std(similarity['semantic'])
        print(f"\t\tTemporal: {temporal_mean:.2f} +/- {temporal_std:.2f}, Semantic: {semantic_mean:.2f} +/- {semantic_std:.2f}, Total: {total_mean:.2f} +/- {total_std:.2f}")
    print("______________________________________________________________________")

In [ ]:
env_name = "Door"
query_type = "video"
npresultspath = GPT_OUTPUT_PATH+f"/{query_type}/{env_name}"
similarity = np.load(f"{npresultspath}_similarity_results.npy", allow_pickle=True)